# Notebook 1: Carga de Datos

## Objetivo
Este notebook se encarga exclusivamente de descargar y almacenar los datos necesarios para toda la práctica.

## Estructura
1. Imports y Configuración
2. Verificación del Archivo desde Google Drive
3. Carga del Dataset
4. Separación del Benchmark
5. Guardado del Dataset Completo

## 1. Imports y Configuración

In [2]:
import pandas as pd

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Constantes
DATA_RAW_DIR = '../datos/raw'
BENCHMARK_TICKER = 'SPY'
START_DATE = '2015-01-01'

print('Librerías importadas correctamente')

Librerías importadas correctamente


## 2. Verificación del Archivo desde Google Drive

**Importante:** El archivo debe descargarse manualmente desde Google Drive y colocarse en la carpeta `datos/raw/` con el nombre `historical_prices.parquet`.

Enlace: https://drive.google.com/file/d/1nvubXdAu0EONlrP_yrURZbnPhBQ-uDaB/view?usp=sharing

In [3]:
# Ruta del archivo de datos
# Buscar el archivo en el directorio raíz o en datos/raw
possible_paths = [
    '../sp500_history.parquet',
    'sp500_history.parquet',
    '../datos/raw/historical_prices.parquet',
    '../datos/raw/sp500_history.parquet',
]

downloaded_file_path = None
for path in possible_paths:
    try:
        test_df = pd.read_parquet(path)
        downloaded_file_path = path
        print(f'Archivo encontrado en: {downloaded_file_path}')
        del test_df
        break
    except (FileNotFoundError, OSError, Exception):
        continue

if downloaded_file_path is None:
    raise FileNotFoundError(
        'Archivo de datos no encontrado. '
        'Por favor, asegúrate de que el archivo parquet esté disponible.'
    )

FileNotFoundError: Archivo de datos no encontrado. Por favor, asegúrate de que el archivo parquet esté disponible.

## 3. Carga del Dataset

In [ ]:
def load_parquet_dataset(file_path):
    """
    Carga un dataset desde un archivo Parquet.
    
    Parámetros:
    -----------
    file_path : str
        Ruta al archivo Parquet
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con los datos cargados
    """
    try:
        print(f'Cargando dataset desde: {file_path}')
        df = pd.read_parquet(file_path)
        print('Dataset cargado correctamente')
        return df
    except Exception as e:
        print(f'Error al cargar el archivo: {str(e)}')
        raise


# Cargar el dataset completo
dataset = load_parquet_dataset(downloaded_file_path)

# Verificar estructura básica
print('\n=== INFORMACIÓN DEL DATASET ===')
print(f'Shape del dataset: {dataset.shape}')
print(f'Columnas: {len(dataset.columns)}')
print(f'Filas: {len(dataset)}')

# Verificar índice de fechas
if not isinstance(dataset.index, pd.DatetimeIndex):
    print('Convirtiendo índice a DatetimeIndex...')
    dataset.index = pd.to_datetime(dataset.index)

# Ordenar por fecha
dataset = dataset.sort_index()

# Mostrar rango de fechas
print(f'\nRango de fechas disponible:')
print(f'  Fecha inicio: {dataset.index.min()}')
print(f'  Fecha fin: {dataset.index.max()}')

# Contar tickers únicos
if isinstance(dataset.columns, pd.MultiIndex):
    tickers = dataset.columns.get_level_values(0).unique()
else:
    tickers = dataset.columns.unique()

print(f'\nNúmero de tickers únicos: {len(tickers)}')

# Mostrar primeras filas para verificación
print('\n=== PRIMERAS FILAS DEL DATASET ===')
print(dataset.head())

## 4. Separación del Benchmark

In [ ]:
def extract_benchmark_data(df, benchmark_ticker):
    """
    Extrae los datos del benchmark del dataset completo.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        Dataset completo con todos los activos
    benchmark_ticker : str
        Símbolo del ticker del benchmark (ej: 'SPY')
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con datos del benchmark
    """
    # Verificar si el dataset tiene MultiIndex en columnas
    if isinstance(df.columns, pd.MultiIndex):
        # Buscar el ticker en el primer nivel
        if benchmark_ticker in df.columns.get_level_values(0):
            benchmark_data = df[benchmark_ticker]
        else:
            raise ValueError(
                f'Benchmark {benchmark_ticker} no encontrado en el dataset'
            )
    else:
        # Columnas simples
        if benchmark_ticker in df.columns:
            benchmark_data = df[[benchmark_ticker]]
        else:
            # Intentar buscar con diferentes variaciones
            possible_names = [
                benchmark_ticker,
                benchmark_ticker.upper(),
                benchmark_ticker.lower(),
            ]
            found = False
            for name in possible_names:
                if name in df.columns:
                    benchmark_data = df[[name]]
                    found = True
                    break
            if not found:
                raise ValueError(
                    f'Benchmark {benchmark_ticker} no encontrado en el dataset'
                )
    
    print(f'Benchmark {benchmark_ticker} extraído correctamente')
    print(f'Shape del benchmark: {benchmark_data.shape}')
    return benchmark_data


# Extraer datos del benchmark SPY
spy_data = extract_benchmark_data(dataset, BENCHMARK_TICKER)

# Guardar benchmark
spy_output_path = f'{DATA_RAW_DIR}/spy_data.parquet'
try:
    spy_data.to_parquet(spy_output_path, engine='pyarrow')
    print(f'Benchmark guardado en: {spy_output_path}')
except (OSError, FileNotFoundError):
    print(f'Error: No se pudo crear el directorio. '
          f'Asegúrate de que {DATA_RAW_DIR} existe.')
    raise

## 5. Guardado del Dataset Completo

In [ ]:
def save_dataset_parquet(df, output_path):
    """
    Guarda un DataFrame en formato Parquet.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a guardar
    output_path : str
        Ruta donde guardar el archivo
    """
    print(f'Guardando dataset en: {output_path}')
    df.to_parquet(output_path, engine='pyarrow')
    print(f'Dataset guardado exitosamente')


# Guardar dataset completo de tickers
tickers_output_path = f'{DATA_RAW_DIR}/tickers_data.parquet'
try:
    save_dataset_parquet(dataset, tickers_output_path)
except (OSError, FileNotFoundError):
    print(f'Error: No se pudo crear el directorio. '
          f'Asegúrate de que {DATA_RAW_DIR} existe.')
    raise

# Resumen final
print('\n=== RESUMEN DEL NOTEBOOK 1 ===')
print('Proceso completado exitosamente')
print(f'\nArchivos generados:')
print(f'  1. {downloaded_file_path}')
print(f'  2. {spy_output_path}')
print(f'  3. {tickers_output_path}')
print(f'\nInformación del dataset:')
print(f'  - Total de tickers: {len(tickers)}')
print(f'  - Rango de fechas: {dataset.index.min()} a {dataset.index.max()}')
print(f'  - Total de registros: {len(dataset)}')
print('\nNotebook 1 completado correctamente')